In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../notion_data/코스피지수.csv')

In [5]:
df = df.dropna()

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2459 entries, 0 to 2467
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   변환      2459 non-null   object
 1   원자료     2459 non-null   object
dtypes: object(2)
memory usage: 57.6+ KB


In [6]:
df2 = df.copy()

In [7]:
df = df.rename(columns={'변환' : '날짜'})

In [8]:
df['날짜'] = pd.to_datetime(df['날짜'])
df = df.sort_values('날짜').reset_index(drop=True)

In [3]:
df.columns

Index(['변환', '원자료'], dtype='object')

In [9]:
df.head()

,날짜,원자료
0,2014-12-30,"1,915.59"
1,2015-01-02,"1,926.44"
2,2015-01-05,"1,915.75"
3,2015-01-06,"1,882.45"
4,2015-01-07,"1,883.83"


In [10]:
import pandas as pd

# 예시 데이터프레임 (실제 df를 사용하시면 됩니다)
print("변환 전 '원자료' 컬럼 타입:")
print(df['원자료'].dtype)
print("\n변환 전 '원자료' 컬럼 내용:")
print(df['원자료'])

# 1. 쉼표(,) 제거: replace() 함수로 쉼표를 빈 문자열로 바꿔줍니다.
df['원자료'] = df['원자료'].str.replace(',', '')

# 2. 숫자로 변환: pd.to_numeric()을 사용하여 숫자로 변환합니다.
#    errors='coerce' 옵션은 숫자로 변환할 수 없는 값(예: 'N/A')을 NaN(Not a Number)으로 처리하게 합니다.
df['원자료'] = pd.to_numeric(df['원자료'], errors='coerce')

print("\n--- 변환 후 ---")
print("변환 후 '원자료' 컬럼 타입:")
print(df['원자료'].dtype)
print("\n변환 후 '원자료' 컬럼 내용:")
print(df['원자료'])

변환 전 '원자료' 컬럼 타입:
object

변환 전 '원자료' 컬럼 내용:
0       1,915.59
1       1,926.44
2       1,915.75
3       1,882.45
4       1,883.83
          ...   
2454    2,442.01
2455    2,440.52
2456    2,429.67
2457    2,404.77
2458    2,399.49
Name: 원자료, Length: 2459, dtype: object

--- 변환 후 ---
변환 후 '원자료' 컬럼 타입:
float64

변환 후 '원자료' 컬럼 내용:
0       1915.59
1       1926.44
2       1915.75
3       1882.45
4       1883.83
         ...   
2454    2442.01
2455    2440.52
2456    2429.67
2457    2404.77
2458    2399.49
Name: 원자료, Length: 2459, dtype: float64


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2459 entries, 0 to 2458
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      2459 non-null   datetime64[ns]
 1   원자료     2459 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 38.6 KB


In [28]:
df.head()

,원자료,일자순번,기간구간,년월
날짜,,,,
2014-12-30,1915.59,1,1~5일,2014-12
2015-01-02,1926.44,1,1~5일,2015-01
2015-01-05,1915.75,2,1~5일,2015-01
2015-01-06,1882.45,3,1~5일,2015-01
2015-01-07,1883.83,4,1~5일,2015-01


In [ ]:
import pandas as pd

# 날짜 컬럼을 datetime으로 변환
df['날짜'] = pd.to_datetime(df['날짜'])

# 날짜를 인덱스로 설정하고 정렬
df = df.set_index('날짜').sort_index()

# 월별로 일자 순번 부여
df['일자순번'] = df.groupby(df.index.to_period('M')).cumcount() + 1

# 기간구간 함수 정의
def 기간구간(day):
    if 1 <= day <= 5:
        return '1~5일'
    elif 6 <= day <= 10:
        return '6~10일'
    elif 11 <= day <= 15:
        return '11~15일'
    elif 16 <= day <= 20:
        return '16~20일'
    else:
        return '기타'

df['기간구간'] = df['일자순번'].apply(기간구간)

# 년월 컬럼 생성
df['년월'] = df.index.to_period('M')

# 일별 수익률 계산 (원자료의 전일 대비 변화율)
df['일별 수익률'] = df['원자료'].pct_change()

# 월별 + 기간구간별 표준편차 계산
df['수익률_표준편차'] = df.groupby(['년월', '기간구간'])['일별 수익률'].transform('std')

# 최종 결과
result = df[['원자료', '일별 수익률', '일자순번', '기간구간', '년월', '수익률_표준편차']]
print(result)


                원자료    일별 수익률  일자순번    기간구간       년월  수익률_표준편차
날짜                                                            
2014-12-30  1915.59       NaN     1    1~5일  2014-12       NaN
2015-01-02  1926.44  0.005664     1    1~5일  2015-01  0.010974
2015-01-05  1915.75 -0.005549     2    1~5일  2015-01  0.010974
2015-01-06  1882.45 -0.017382     3    1~5일  2015-01  0.010974
2015-01-07  1883.83  0.000733     4    1~5일  2015-01  0.010974
...             ...       ...   ...     ...      ...       ...
2024-12-23  2442.01  0.015748    16  16~20일  2024-12  0.009713
2024-12-24  2440.52 -0.000610    17  16~20일  2024-12  0.009713
2024-12-26  2429.67 -0.004446    18  16~20일  2024-12  0.009713
2024-12-27  2404.77 -0.010248    19  16~20일  2024-12  0.009713
2024-12-30  2399.49 -0.002196    20  16~20일  2024-12  0.009713

[2459 rows x 6 columns]


In [30]:
df.head()

,원자료,일자순번,기간구간,년월,일별 수익률,수익률_표준편차
날짜,,,,,,
2014-12-30,1915.59,1,1~5일,2014-12,NaN,NaN
2015-01-02,1926.44,1,1~5일,2015-01,0.005664,0.010974
2015-01-05,1915.75,2,1~5일,2015-01,-0.005549,0.010974
2015-01-06,1882.45,3,1~5일,2015-01,-0.017382,0.010974
2015-01-07,1883.83,4,1~5일,2015-01,0.000733,0.010974


In [32]:
import pandas as pd

# 날짜 컬럼을 datetime으로 변환하고 인덱스로 설정
# df['날짜'] = pd.to_datetime(df['날짜'])
# df = df.set_index('날짜')
# df = df.sort_index()

# 월별로 일자순번 붙이기 (월별로 1부터 순번)
df['일자순번'] = df.groupby(df.index.to_period('M')).cumcount() + 1

# 기간구간 함수 정의
def 기간구간(day):
    if 1 <= day <= 5:
        return '1~5일'
    elif 6 <= day <= 10:
        return '6~10일'
    elif 11 <= day <= 15:
        return '11~15일'
    elif 16 <= day <= 20:
        return '16~20일'
    else:
        return '기타'

df['기간구간'] = df['일자순번'].apply(기간구간)

# 년월 컬럼 추가
df['년월'] = df.index.to_period('M')

# 월별, 기간구간별 표준편차 계산 및 추가
df['수익률_표준편차'] = df.groupby(['년월', '기간구간'])['일별 수익률'].transform('std')

# 결과 추출 및 인덱스 이름 변경
result = df[['원자료', '일별 수익률', '일자순번', '기간구간', '년월', '수익률_표준편차']].copy()
result.index.name = '변환'

print(result)


                원자료    일별 수익률  일자순번    기간구간       년월  수익률_표준편차
변환                                                            
2014-12-30  1915.59       NaN     1    1~5일  2014-12       NaN
2015-01-02  1926.44  0.005664     1    1~5일  2015-01  0.010974
2015-01-05  1915.75 -0.005549     2    1~5일  2015-01  0.010974
2015-01-06  1882.45 -0.017382     3    1~5일  2015-01  0.010974
2015-01-07  1883.83  0.000733     4    1~5일  2015-01  0.010974
...             ...       ...   ...     ...      ...       ...
2024-12-23  2442.01  0.015748    16  16~20일  2024-12  0.009713
2024-12-24  2440.52 -0.000610    17  16~20일  2024-12  0.009713
2024-12-26  2429.67 -0.004446    18  16~20일  2024-12  0.009713
2024-12-27  2404.77 -0.010248    19  16~20일  2024-12  0.009713
2024-12-30  2399.49 -0.002196    20  16~20일  2024-12  0.009713

[2459 rows x 6 columns]


In [33]:
df.head(30)

,원자료,일자순번,기간구간,년월,일별 수익률,수익률_표준편차
날짜,,,,,,
2014-12-30,1915.59,1,1~5일,2014-12,NaN,NaN
2015-01-02,1926.44,1,1~5일,2015-01,0.005664,0.010974
2015-01-05,1915.75,2,1~5일,2015-01,-0.005549,0.010974
2015-01-06,1882.45,3,1~5일,2015-01,-0.017382,0.010974
2015-01-07,1883.83,4,1~5일,2015-01,0.000733,0.010974
2015-01-08,1904.65,5,1~5일,2015-01,0.011052,0.010974
2015-01-09,1924.70,6,6~10일,2015-01,0.010527,0.005405
2015-01-12,1920.95,7,6~10일,2015-01,-0.001948,0.005405
2015-01-13,1917.14,8,6~10일,2015-01,-0.001983,0.005405


In [38]:
# 기존 df에서 필요한 컬럼 선택
pivot_df = df[['수익률_표준편차', '기간구간']].copy()

# 컬럼 이름 변경 (출력 포맷에 맞게)
pivot_df.columns = ['수익률_표준편차', '구간']

# 각 기간구간 이름을 열 이름 형식에 맞게 변환
구간명변환 = {
    '1~5일': '코스피수익률표준편차(1~5)',
    '6~10일': '코스피수익률표준편차(6~10)',
    '11~15일': '코스피수익률표준편차(11~15)',
    '16~20일': '코스피수익률표준편차(16~20)',
    '기타': '코스피수익률표준편차(기타)'
}
pivot_df['구간'] = pivot_df['구간'].map(구간명변환)

# 피벗 테이블로 변환: 날짜별로 각 구간별 표준편차 열로 나열
pivot_table = pivot_df.pivot(columns='구간', values='수익률_표준편차')

# 원래 인덱스가 날짜이므로 유지
pivot_table.index.name = '날짜'

# 결과 확인
print(pivot_table.head())


구간          코스피수익률표준편차(11~15)  코스피수익률표준편차(16~20)  코스피수익률표준편차(1~5)  \
날짜                                                                  
2014-12-30                NaN                NaN              NaN   
2015-01-02                NaN                NaN         0.010974   
2015-01-05                NaN                NaN         0.010974   
2015-01-06                NaN                NaN         0.010974   
2015-01-07                NaN                NaN         0.010974   

구간          코스피수익률표준편차(6~10)  코스피수익률표준편차(기타)  
날짜                                            
2014-12-30               NaN             NaN  
2015-01-02               NaN             NaN  
2015-01-05               NaN             NaN  
2015-01-06               NaN             NaN  
2015-01-07               NaN             NaN  


In [39]:
# 원하는 컬럼 순서
desired_columns = [
    '코스피수익률표준편차(1~5)',
    '코스피수익률표준편차(6~10)',
    '코스피수익률표준편차(11~15)',
    '코스피수익률표준편차(16~20)'
]

# 누락된 컬럼이 있을 수도 있으므로 존재하는 컬럼만 추출
existing_columns = [col for col in desired_columns if col in pivot_table.columns]

# 순서대로 재정렬
pivot_table = pivot_table[existing_columns]

# 결과 확인
print(pivot_table.head())


구간          코스피수익률표준편차(1~5)  코스피수익률표준편차(6~10)  코스피수익률표준편차(11~15)  \
날짜                                                                 
2014-12-30              NaN               NaN                NaN   
2015-01-02         0.010974               NaN                NaN   
2015-01-05         0.010974               NaN                NaN   
2015-01-06         0.010974               NaN                NaN   
2015-01-07         0.010974               NaN                NaN   

구간          코스피수익률표준편차(16~20)  
날짜                             
2014-12-30                NaN  
2015-01-02                NaN  
2015-01-05                NaN  
2015-01-06                NaN  
2015-01-07                NaN  


In [40]:
pivot_table.head(30)

구간,코스피수익률표준편차(1~5),코스피수익률표준편차(6~10),코스피수익률표준편차(11~15),코스피수익률표준편차(16~20)
날짜,,,,
2014-12-30,NaN,NaN,NaN,NaN
2015-01-02,0.010974,NaN,NaN,NaN
2015-01-05,0.010974,NaN,NaN,NaN
2015-01-06,0.010974,NaN,NaN,NaN
2015-01-07,0.010974,NaN,NaN,NaN
2015-01-08,0.010974,NaN,NaN,NaN
2015-01-09,NaN,0.005405,NaN,NaN
2015-01-12,NaN,0.005405,NaN,NaN
2015-01-13,NaN,0.005405,NaN,NaN


In [44]:
pivot_table.to_csv('final_코스피.csv', encoding='utf-8-sig')

In [ ]:
# # '날짜'를 인덱스로 설정
# df = df.set_index('날짜')

# # 일별 수익률 계산 (pct_change는 전일 대비 퍼센트 변동)
# returns = df.pct_change()

# # 결과 저장용 DataFrame 생성
# result = pd.DataFrame(index=df.index)

# # (shift, 라벨) 쌍 정의: 
# # shift=1 → -1~-5일, shift=6 → -6~-10일, shift=11 → -11~-15일, shift=16 → -16~-20일
# windows = [
#     (1, '(-1~-5)'),
#     (6, '(-6~-10)'),
#     (11, '(-11~-15)'),
#     (16, '(-16~-20)')
# ]

# # 각 컬럼에 대해 네 구간별 표준편차 계산
# for col in df.columns:
#     for shift, label in windows:
#         result[f"{col}{label}"] = (
#             returns[col]
#             .shift(shift)              # 과거 shift 일만큼 뒤로
#             .rolling(window=5)         # 5일간
#             .std()                     # 표준편차
#         )

# # 인덱스를 '날짜' 컬럼으로 되돌리고
# result = result.reset_index()

# # 결과 확인
# print(result.head())
# # 필요시 to_csv 등으로 저장



          날짜  원자료(-1~-5)  원자료(-6~-10)  원자료(-11~-15)  원자료(-16~-20)
0 2014-12-30         NaN          NaN           NaN           NaN
1 2015-01-02         NaN          NaN           NaN           NaN
2 2015-01-05         NaN          NaN           NaN           NaN
3 2015-01-06         NaN          NaN           NaN           NaN
4 2015-01-07         NaN          NaN           NaN           NaN


In [ ]:
result = result.rename(columns={
    '원자료(-1~-5)' : '코스피수익률표준편차(-1~-5)',
    '원자료(-6~-10)' : '코스피수익률표준편차(-6~-10)',
    '원자료(-11~-15)' : '코스피수익률표준편차(-11~-15)',
    '원자료(-16~-20)' : '코스피수익률표준편차(-16~-20)',
    '기타' : '기타'
})

# result = result.rename(columns={
#     '코스피(-1~-5)' : '코스피수익률표준편차(-1~-5)',
#     '코스피(-6~-10)' : '코스피수익률표준편차(-6~-10)',
#     '코스피(-11~-15)' : '코스피수익률표준편차(-11~-15)',
#     '코스피(-16~-20)' : '코스피수익률표준편차(-16~-20)',
#     '기타' : '기타'
# })


In [46]:
result.head(30)

,날짜,코스피수익률표준편차(-1~-5),코스피수익률표준편차(-6~-10),코스피수익률표준편차(-11~-15),코스피수익률표준편차(-16~-20)
0,2014-12-30,NaN,NaN,NaN,NaN
1,2015-01-02,NaN,NaN,NaN,NaN
2,2015-01-05,NaN,NaN,NaN,NaN
3,2015-01-06,NaN,NaN,NaN,NaN
4,2015-01-07,NaN,NaN,NaN,NaN
5,2015-01-08,NaN,NaN,NaN,NaN
6,2015-01-09,0.010974,NaN,NaN,NaN
7,2015-01-12,0.011899,NaN,NaN,NaN
8,2015-01-13,0.011594,NaN,NaN,NaN
9,2015-01-14,0.006589,NaN,NaN,NaN


In [45]:
result.to_csv('코스피수익률표준편차.csv', index=False)

In [ ]:
# result.to_csv('result.csv', encoding='cp949')

In [64]:
macro_df = pd.read_csv('../data/macro_data.csv')

In [65]:
macro_df.head()

,회사명,stock_code,상장일,kospi200(-20),nasdaq(-20),putcall(-20),VIXCLS,미국10년물국채금리수익률(-20),시장금리국고채3년,환율변동률(-20),...,육우(-11~-15),육우(-16~-20),미국_대두(-1~-5),미국_대두(-6~-10),미국_대두(-11~-15),미국_대두(-16~-20),코스닥수익률표준편차(-1~-5),코스닥수익률표준편차(-6~-10),코스닥수익률표준편차(-11~-15),코스닥수익률표준편차(-16~-20)
0,포시에스,189690,2015-02-11,-0.053391,0.027060,0.056075,16.96,0.080323,2.051,0.014332,...,0.016163,0.005766,0.006163,0.016230,0.004037,0.006819,0.010184,0.006112,0.010859,0.007335
1,세화아이엠씨,145210,2015-03-19,0.049828,0.011804,-0.338983,14.07,-0.096136,1.825,0.028860,...,0.011177,0.007051,0.009376,0.011863,0.008764,0.009535,0.009558,0.007889,0.006379,0.007637
2,엔에스쇼핑,138250,2015-03-27,0.058506,-0.020181,-0.042553,15.07,0.000000,1.770,0.003184,...,0.003275,0.006386,0.005733,0.011345,0.008212,0.007389,0.011123,0.007663,0.011057,0.006264
3,유지인트,195990,2015-04-13,-0.016436,0.025498,-0.070707,13.94,-0.060270,1.732,-0.028163,...,0.004451,0.003749,0.008171,0.004182,0.008052,0.002805,0.004129,0.005452,0.009951,0.008684
4,제노포커스,187420,2015-05-29,0.028846,0.031683,0.324324,13.84,0.010890,1.745,0.030247,...,0.001259,0.002430,0.002637,0.005841,0.006451,0.008910,0.010341,0.005583,0.009270,0.007178


In [66]:
# 1) Convert both columns to datetime
macro_df['상장일'] = pd.to_datetime(macro_df['상장일'])
result['날짜'] = pd.to_datetime(result['날짜'])

# 2) Sort both DataFrames by their date keys
merged_full = macro_df.sort_values('상장일').reset_index(drop=True)
result = result.sort_values('날짜').reset_index(drop=True)

merged_full = pd.merge_asof(
    merged_full,
    result,
    left_on='상장일',
    right_on='날짜',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['날짜'])

In [69]:
merged_full.to_csv('macro_data.csv', encoding='utf-8-sig', index=False)


In [68]:
merged_full.columns

Index(['회사명', 'stock_code', '상장일', 'kospi200(-20)', 'nasdaq(-20)',
       'putcall(-20)', 'VIXCLS', '미국10년물국채금리수익률(-20)', '시장금리국고채3년',
       '환율변동률(-20)',
       ...
       '미국_대두(-11~-15)', '미국_대두(-16~-20)', '코스닥수익률표준편차(-1~-5)',
       '코스닥수익률표준편차(-6~-10)', '코스닥수익률표준편차(-11~-15)', '코스닥수익률표준편차(-16~-20)',
       '코스피수익률표준편차(-1~-5)', '코스피수익률표준편차(-6~-10)', '코스피수익률표준편차(-11~-15)',
       '코스피수익률표준편차(-16~-20)'],
      dtype='object', length=164)